### Financial Crime & Suspicious Matter Reporting - LLM-based Agentic Solution

This sample code has been put together to demonstrate the capabilities of LLMs in being able to detect money laundering and other financial crime related activities. The example below demonstrates the use of LLM-based agents to identify suspicious activities from a call-centre log and draft a suspicious matter report.

The multi-agent example published by Microsoft has been used as a reference for this notebook - https://github.com/Azure-Samples/azureai-samples/blob/main/scenarios/Assistants/multi-agent/multi-agent.ipynb


In [164]:
# Uncomment line below to install dependencies; re-comment after installation.
# %pip install -r requirements.txt

In [165]:
import os
import time
from dotenv import load_dotenv
from openai import AzureOpenAI
from openai.types.beta import Thread
from openai.types.beta import Assistant

load_dotenv()

assistant_client = AzureOpenAI(
    api_key=os.getenv("GPT4_AZURE_OPENAI_KEY"),
    api_version=os.getenv("GPT4_AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv(
        "GPT4_AZURE_OPENAI_ENDPOINT"
    ),
)

assistant_deployment_name = os.getenv(
    "GPT4_DEPLOYMENT_NAME"
)

In [166]:
name_reviewer = "reviewer_assistant"
instructions_reviewer = """You are an assistant that understands guidelines published by AUSTRAC for detecting anti-money laundering and financial crime activities. Your job is to review the log from a call centre interaction between an agent and a customer and detect any suspicious activities. Generate a detailed summary with specific points on the suspicious activities."""

verbose_output = True

In [167]:
reviewer_assistant = assistant_client.beta.assistants.create(
    name=name_reviewer, instructions=instructions_reviewer, model=assistant_deployment_name
)

In [168]:
name_reportwriter = "reportwriter_assistant"
instructions_reportwriter = """You are an assistant that receives a list of suspicious activities and then drafts a Suspicious Matter Report (SMR) for it based on AUSTRAC's published guidelines. Ensure the report aligns with the format required by AUSTRAC."""

verbose_output = True

In [169]:
reportwriter_assistant = assistant_client.beta.assistants.create(
    name=name_reportwriter, instructions=instructions_reportwriter, model=assistant_deployment_name
)

In [170]:
name_approver = "approver_assistant"
instructions_approver = """You are an assistant that understands AUSTRAC's published guidelines for Suspicious Matter Reporting (SMR). Your job is to review a draft SMR and ensure it captures all suspicious activities that were previously identified."""

verbose_output = True

In [171]:
approver_assistant = assistant_client.beta.assistants.create(
    name=name_approver, instructions=instructions_approver, model=assistant_deployment_name
)

In [172]:
thread = assistant_client.beta.threads.create()

In [173]:
user_message = f"""
    << Add call-centre log here >>
    """

In [174]:
def run_assistant(assistant_id, thread_id, instructions):
    run = assistant_client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions=instructions,
    )

    while True:
        run = assistant_client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

        if run.status=="completed":
            print("Done!")
            break
        else:
            print("Task executing ...")
            time.sleep(5)

In [175]:
def print_messages(thread_id):
    messages = assistant_client.beta.threads.messages.list(
      thread_id=thread_id
    )

    for msg in messages.data:
        for content_item in msg.content:
            print(content_item.text.value)

In [176]:
run_assistant(
    reviewer_assistant.id, 
    thread.id, 
    f"Review the conversation log provided and draft a summary of suspicious activities - " + user_message
)

Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Done!


In [177]:
print_messages(thread.id)

Summary of Suspicious Activities:

1. Lack of Standard Identification: Alex Tracer, the customer, was unable to provide a standard form of identification such as an ATM card number or driver’s license, claiming to have lost the former and to be in the process of acquiring the latter. He eventually provided a passport ID for identification.

2. Inconsistent Information: Alex Tracer initially provided an incorrect account number, which he then corrected. This could indicate a lack of familiarity with the account details or an attempt to deceive.

3. No Phone for Verification: Despite the bank operator, Adam Krivy, attempting to complete an authentication process with a verification code sent to a mobile number, Alex claimed not to have a phone, thus avoiding this security measure.

4. Urgency and Pressure: The customer repeatedly stressed the urgency of the transfers, claiming lives were at risk and expressing a strong desire to avoid bank bureaucracy, thereby trying to rush the process 

In [178]:
run_assistant(
    reportwriter_assistant.id, 
    thread.id, 
    f"Review the summary of suspicious activities written by the reviewer_assistant assitant and draft a Suspicious Matter Report (SMR)"
)

Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Done!


In [179]:
print_messages(thread.id)

SUSPICIOUS MATTER REPORT (SMR)

I. Reporter Information:
- Financial Institution: [Your Bank's Name]
- Name of Submitter: [Your Full Name]
- Position/Title: [Your Position/Title]
- Contact Information: [Your Direct Phone Line and/or Email]
- Submission Date: [Date of Submission]

II. Subject Information:
- Name of Suspect: Alex Tracer (if alias is suspected, please note)
- Address: [Not Provided/Available]
- Date of Birth: [Not Provided/Available]
- Identification Type/Number: Passport ID [Number if available]
- Occupation/Industry: [Not Provided/Available]
- Account Number(s): [Initial Incorrect Number], [Corrected Account Number]
  
III. Details of Suspicious Activities:
A summary of suspicious activities is as follows:
1. Identification Issue: Subject failed to provide standard identification initially; eventually used a passport for ID.
2. Account Number Discrepancy: Incorrect account number provided, later corrected.
3. Phone Number for Verification: Subject claimed to lack a phon

In [180]:
run_assistant(
    approver_assistant.id, 
    thread.id, 
    f"Check the Suspicious Matter Report (SMR) created by the reportwriter_assistant assistant and make sure all points mentioned in the summary of suspicious activities generated by the reviewer_assistant assitant are captured in the final report."
)

Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Task executing ...
Done!


In [181]:
print_messages(thread.id)

To ensure that the Suspicious Matter Report (SMR) captures all points mentioned in the summary:

**Points in Summary of Suspicious Activities:**
1. Lack of Standard Identification
2. Inconsistent Information
3. No Phone for Verification
4. Urgency and Pressure
5. Large International Transfers
6. Transfers to Charities Without Adequate Information
7. Defensive Response

**Verification Against SMR:**

1. **Lack of Standard Identification:** The SMR addresses this point in Section III, Item 1. "Identification Issue: Subject failed to provide standard identification initially; eventually used a passport for ID."
2. **Inconsistent Information:** Addressed in Section III, Item 2. "Account Number Discrepancy: Incorrect account number provided, later corrected."
3. **No Phone for Verification:** Included in Section III, Item 3. "Phone Number for Verification: Subject claimed to lack a phone, evading a key security control."
4. **Urgency and Pressure:** Mentioned in Section III, Item 4. "Urgent

In [182]:
# Clean-up
response = assistant_client.beta.assistants.delete(reviewer_assistant.id)
response = assistant_client.beta.assistants.delete(reportwriter_assistant.id)